<p><h1>Capstone Project - The Battle of the Neighborhoods</h1>
<h3>Applied Data Science Capstone</h3></p>

<h2>Business Problem</h2>

In this project, we will leverage location data from foursquare to find the best area/s to open a new restaurant in the city of New York. We will also be looking at what type of restuarant should be opened. We will do this by finding areas that are not overcrowded with restaurants already and looking at the trends of other
areas to figure out what type of restaurant we should open.

We will outline the what we believe to be the best area/s based on the data and outline why we believe these areas are good locations.


<h2>Data</h2>
Based on our problem, the factors that will impact our decision are:
<ul>
    <li>The number of restaurants in a neighborhood</li>
    <li>The type of restaurants in a naighborhood</li>
    <li>The ratio of different types of resturants in a specific area</li>
</ul>

The following data sources will be needed to generate the required information:
<ul>
    <li>New York City Neighborhoods Dataset(Used Previously in Labs)</li>
    <li>Foursquare</li>
</ul>

<h2>IMPORT RELEVANT LIBRARIES</h2>

In [6]:
import numpy as np 
import pandas as pd 
import json 

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

ModuleNotFoundError: No module named 'folium'

<h2>Neighborhood Candidates</h2>

In [2]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset  #Getting the neighborhoods of new york

<h3>Load the data</h3>

In [11]:
neighborhoods_data = newyork_data['features']
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [12]:
import json
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [13]:
#define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [15]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [101]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [19]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.0               |             py_0          26 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         673 KB

The following NEW packages will be INSTALLED:

    altair:  4.0.1-py_0 conda-forge
    branca:  0.4.0-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


branca-0.4.0         | 26 KB     | ##################################### | 1

In [105]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

In [25]:
boroughs = neighborhoods.groupby('Borough').count()
boroughs

,Neighborhood,Latitude,Longitude
Borough,,,
Bronx,104,104,104
Brooklyn,140,140,140
Manhattan,80,80,80
Queens,162,162,162
Staten Island,126,126,126


In the city of New York there are five Boroughs, that being Bronx, Brooklyn, Manhattan, Queens and Staten Island. We will now breakdown the data into these five boroughs to analyze further.

In [111]:
Manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
Queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
Bronx_data = neighborhoods[neighborhoods['Borough'] == 'Bronx'].reset_index(drop=True)
Brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
Staten_data = neighborhoods[neighborhoods['Borough'] == 'Staten Island'].reset_index(drop=True)

,Borough,Neighborhood,Latitude,Longitude
0,Staten Island,St. George,40.644982,-74.079353
1,Staten Island,New Brighton,40.640615,-74.087017
2,Staten Island,Stapleton,40.626928,-74.077902
3,Staten Island,Rosebank,40.615305,-74.069805
4,Staten Island,West Brighton,40.631879,-74.107182


<h2>Borough Data</h2>

In [106]:
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Manhattan_data['Latitude'], Manhattan_data['Longitude'], Manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
for lat, lng, label in zip(Bronx_data['Latitude'], Bronx_data['Longitude'], Bronx_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)
for lat, lng, label in zip(Queens_data['Latitude'], Queens_data['Longitude'], Queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
for lat, lng, label in zip(Brooklyn_data['Latitude'], Brooklyn_data['Longitude'], Brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)
for lat, lng, label in zip(Staten_data['Latitude'], Staten_data['Longitude'], Staten_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
    
map_york

In [38]:
CLIENT_ID = '41QLDPLFWYG0MKJPWFEDMJJTF1HE4SRPYDVJ12BKKDYS0Z1C' 
CLIENT_SECRET = 'B10E2QARDMEY3H2ZQ3N4MJKZGI2MW3BZ30GHE402UE4QYGHL' 
VERSION = '20180605' 
LIMIT = 100

In [39]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [40]:
Manhattan_venues = getNearbyVenues(names=Manhattan_data['Neighborhood'],
                                   latitudes=Manhattan_data['Latitude'],
                                   longitudes=Manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards
Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyve

In [115]:
Bronx_venues = getNearbyVenues(names=Bronx_data['Neighborhood'],
                                   latitudes=Bronx_data['Latitude'],
                                   longitudes=Bronx_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Claremont Village
Concourse Village
Mount Eden
Mount Hope
Bronxdale
Allerton
Kingsbridge Heights
Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square

In [108]:
Brooklyn_venues = getNearbyVenues(names=Brooklyn_data['Neighborhood'],
                                   latitudes=Brooklyn_data['Latitude'],
                                   longitudes=Brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heig

In [109]:
Queens_venues = getNearbyVenues(names=Queens_data['Neighborhood'],
                                   latitudes=Queens_data['Latitude'],
                                   longitudes=Queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge
Astoria
Woodside
Jackson Heights
Elmhur

In [119]:
Staten_venues = getNearbyVenues(names=Staten_data['Neighborhood'],
                                   latitudes=Staten_data['Latitude'],
                                   longitudes=Staten_data['Longitude']
                                  )

St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsville
Silver Lake
Sunnyside
Park Hill
Westerleigh
Graniteville
Arlington
Arrochar
Grasmere
Old Town
Dongan Hills
Midland Beach
Grant City
New Dorp Beach
Bay Terrace
Huguenot
Pleasant Plains
Butler Manor
Charleston
Rossville
Arden Heights
Greenridge
Heartland Village
Chelsea
Bloomfield
Bulls Head
Richmond Town
Shore Acres
Clifton
Concord
Emerson Hill
Randall Manor
Howland Hook
Elm Park
Manor Heights
Willowbrook
Sandy Ground
Egbertville
Prince's Bay
Lighthouse Hill
Richmond Valley
Fox Hills
St. George
New Brighton
Stapleton
Rosebank
West Brighton
Grymes Hill
Todt Hill
South Beach
Port Richmond
Mariner's Harbor
Port Ivory
Castleton Corners
New Springville
Travis
New Dorp
Oakwood
Great Kills
Eltingville
Annadale
Woodrow
Tottenville
Tompkinsvi

In [123]:
Manhattan_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,194,194,194,194,194,194
Carnegie Hill,200,200,200,200,200,200
Central Harlem,86,86,86,86,86,86
Chelsea,200,200,200,200,200,200
Chinatown,200,200,200,200,200,200


In [121]:
Bronx_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Allerton,56,56,56,56,56,56
Baychester,42,42,42,42,42,42
Bedford Park,76,76,76,76,76,76
Belmont,198,198,198,198,198,198
Bronxdale,24,24,24,24,24,24


In [122]:
Brooklyn_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,84,84,84,84,84,84
Bay Ridge,178,178,178,178,178,178
Bedford Stuyvesant,52,52,52,52,52,52
Bensonhurst,70,70,70,70,70,70
Bergen Beach,14,14,14,14,14,14


In [124]:
Staten_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Annadale,20,20,20,20,20,20
Arden Heights,10,10,10,10,10,10
Arlington,12,12,12,12,12,12
Arrochar,38,38,38,38,38,38
Bay Terrace,20,20,20,20,20,20


In [125]:
Queens_venues.groupby('Neighborhood').count().head()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,34,34,34,34,34,34
Astoria,200,200,200,200,200,200
Astoria Heights,26,26,26,26,26,26
Auburndale,36,36,36,36,36,36
Bay Terrace,74,74,74,74,74,74


In [135]:

Manhattan_onehot = pd.get_dummies(Manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
Bronx_onehot = pd.get_dummies(Bronx_venues[['Venue Category']], prefix="", prefix_sep="")
Brooklyn_onehot = pd.get_dummies(Brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")
Queens_onehot = pd.get_dummies(Queens_venues[['Venue Category']], prefix="", prefix_sep="")
Staten_onehot = pd.get_dummies(Staten_venues[['Venue Category']], prefix="", prefix_sep="")


Manhattan_onehot['Neighborhood'] = Manhattan_venues['Neighborhood'] 
Bronx_onehot['Neighborhood'] = Bronx_venues['Neighborhood'] 
Brooklyn_onehot['Neighborhood'] = Brooklyn_venues['Neighborhood'] 
Queens_onehot['Neighborhood'] = Queens_venues['Neighborhood'] 
Staten_onehot['Neighborhood'] = Staten_venues['Neighborhood'] 

fixed_columns = [Manhattan_onehot.columns[-1]] + list(Manhattan_onehot.columns[:-1])
Manhattan_onehot = Manhattan_onehot[fixed_columns]

fixed_columns = [Bronx_onehot.columns[-1]] + list(Bronx_onehot.columns[:-1])
Bronx_onehot = Bronx_onehot[fixed_columns]

fixed_columns = [Brooklyn_onehot.columns[-1]] + list(Brooklyn_onehot.columns[:-1])
Brooklyn_onehot = Brooklyn_onehot[fixed_columns]

fixed_columns = [Queens_onehot.columns[-1]] + list(Queens_onehot.columns[:-1])
Queens_onehot = Queens_onehot[fixed_columns]

fixed_columns = [Staten_onehot.columns[-1]] + list(Staten_onehot.columns[:-1])
Staten_onehot = Staten_onehot[fixed_columns]


Staten_onehot.head()


,Neighborhood,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,St. George,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [168]:
Manhattan_grouped = Manhattan_onehot.groupby('Neighborhood').mean().reset_index()
Bronx_grouped = Bronx_onehot.groupby('Neighborhood').mean().reset_index()
Brooklyn_grouped = Brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
Staten_grouped = Staten_onehot.groupby('Neighborhood').mean().reset_index()
Queens_grouped = Queens_onehot.groupby('Neighborhood').mean().reset_index()

num_top_venues = 1

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('Venue')
    except:
        columns.append('Venue')


Manhattan_venues_sorted = pd.DataFrame(columns=columns)
Manhattan_venues_sorted['Neighborhood'] = Manhattan_grouped['Neighborhood']

for ind in np.arange(Manhattan_grouped.shape[0]):
    Manhattan_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Manhattan_grouped.iloc[ind, :], num_top_venues)

Bronx_venues_sorted = pd.DataFrame(columns=columns)
Bronx_venues_sorted['Neighborhood'] = Bronx_grouped['Neighborhood']

for ind in np.arange(Bronx_grouped.shape[0]):
    Bronx_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Bronx_grouped.iloc[ind, :], num_top_venues)
    
Brooklyn_venues_sorted = pd.DataFrame(columns=columns)
Brooklyn_venues_sorted['Neighborhood'] = Brooklyn_grouped['Neighborhood']

for ind in np.arange(Brooklyn_grouped.shape[0]):
    Brooklyn_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Brooklyn_grouped.iloc[ind, :], num_top_venues)    
    
Queens_venues_sorted = pd.DataFrame(columns=columns)
Queens_venues_sorted['Neighborhood'] = Queens_grouped['Neighborhood']

for ind in np.arange(Queens_grouped.shape[0]):
    Queens_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Queens_grouped.iloc[ind, :], num_top_venues)
    
Staten_venues_sorted = pd.DataFrame(columns=columns)
Staten_venues_sorted['Neighborhood'] = Staten_grouped['Neighborhood']

for ind in np.arange(Staten_grouped.shape[0]):
    Staten_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Staten_grouped.iloc[ind, :], num_top_venues)
    

Manhattan_venues_sorted.head()

,Neighborhood,Venue
0,Battery Park City,Coffee Shop
1,Carnegie Hill,Coffee Shop
2,Central Harlem,African Restaurant
3,Chelsea,Coffee Shop
4,Chinatown,Chinese Restaurant


In [169]:
Staten_venues_sorted.head()

,Neighborhood,Venue
0,Annadale,American Restaurant
1,Arden Heights,Deli / Bodega
2,Arlington,Bus Stop
3,Arrochar,Deli / Bodega
4,Bay Terrace,Supermarket


In [170]:
Queens_venues_sorted.head()

,Neighborhood,Venue
0,Arverne,Surf Spot
1,Astoria,Bar
2,Astoria Heights,Plaza
3,Auburndale,Discount Store
4,Bay Terrace,Clothing Store


In [171]:
Bronx_venues_sorted.head()

,Neighborhood,Venue
0,Allerton,Pizza Place
1,Baychester,Donut Shop
2,Bedford Park,Mexican Restaurant
3,Belmont,Italian Restaurant
4,Bronxdale,Bank


In [172]:
Brooklyn_venues_sorted.head()

,Neighborhood,Venue
0,Bath Beach,Pharmacy
1,Bay Ridge,Pizza Place
2,Bedford Stuyvesant,Deli / Bodega
3,Bensonhurst,Grocery Store
4,Bergen Beach,Harbor / Marina


In [149]:
pd.value_counts(Bronx_venues_sorted.Venue)

Pizza Place                  11
Bus Station                   5
Pharmacy                      4
Italian Restaurant            3
Spanish Restaurant            2
Deli / Bodega                 2
Park                          2
Grocery Store                 2
Bank                          2
Donut Shop                    2
Fast Food Restaurant          2
Sandwich Place                1
Discount Store                1
Bakery                        1
Soup Place                    1
Harbor / Marina               1
Supermarket                   1
Mobile Phone Shop             1
Chinese Restaurant            1
Intersection                  1
Fish Market                   1
Plaza                         1
Mexican Restaurant            1
Pub                           1
Caribbean Restaurant          1
Latin American Restaurant     1
Name: Venue, dtype: int64

In [150]:
pd.value_counts(Brooklyn_venues_sorted.Venue)

Pizza Place                    11
Deli / Bodega                   7
Bar                             7
Caribbean Restaurant            4
Pharmacy                        4
Grocery Store                   3
Coffee Shop                     3
Bank                            3
Dance Studio                    2
Italian Restaurant              2
Dessert Shop                    1
Women's Store                   1
Spanish Restaurant              1
Gym                             1
Art Gallery                     1
Eastern European Restaurant     1
Spa                             1
Fried Chicken Joint             1
Bus Station                     1
Playground                      1
Yoga Studio                     1
Bus Stop                        1
Restaurant                      1
Donut Shop                      1
Food                            1
Park                            1
Diner                           1
Bakery                          1
Pool                            1
Burger Joint  

In [151]:
pd.value_counts(Staten_venues_sorted.Venue)

Bus Stop               10
Deli / Bodega           7
Italian Restaurant      6
Pizza Place             5
American Restaurant     4
Bar                     2
Pharmacy                2
Clothing Store          2
Coffee Shop             2
Bank                    2
Arcade                  1
Big Box Store           1
Recreation Center       1
Sushi Restaurant        1
Rental Car Location     1
Grocery Store           1
Pool                    1
Chinese Restaurant      1
Steakhouse              1
Shipping Store          1
Brewery                 1
Supermarket             1
Park                    1
Hotel                   1
Beach                   1
Donut Shop              1
Food                    1
Discount Store          1
Food Truck              1
Name: Venue, dtype: int64

In [152]:
pd.value_counts(Queens_venues_sorted.Venue)

Deli / Bodega                      9
Grocery Store                      5
Pizza Place                        5
Beach                              5
Chinese Restaurant                 4
Caribbean Restaurant               4
Park                               3
Bar                                3
Hotel                              3
Bus Station                        3
Pharmacy                           2
Mobile Phone Shop                  2
Bakery                             2
Donut Shop                         2
Korean Restaurant                  2
Bank                               2
Indian Restaurant                  2
Rental Car Location                1
Gym / Fitness Center               1
Department Store                   1
Latin American Restaurant          1
Cosmetics Shop                     1
Surf Spot                          1
Baseball Field                     1
Food & Drink Shop                  1
Bus Stop                           1
Discount Store                     1
I

In [153]:
pd.value_counts(Manhattan_venues_sorted.Venue)

Coffee Shop             7
Italian Restaurant      6
Bar                     3
Park                    3
Sandwich Place          2
American Restaurant     2
Mexican Restaurant      2
African Restaurant      1
Chinese Restaurant      1
Theater                 1
Pizza Place             1
Plaza                   1
Café                    1
Bakery                  1
Exhibit                 1
Gym / Fitness Center    1
Clothing Store          1
Art Gallery             1
Gym                     1
Korean Restaurant       1
Hotel                   1
Yoga Studio             1
Name: Venue, dtype: int64


<h3>Burough Analysis</h3>
According to the analysis of the five boroughs and their neighbourhood, the data suggests that the three most frequently found venues of the five boroughs are as follows:

<ul><h4>Manhattan</h4>
    <li>Coffee Shop</li>
    <li>Italian Restaurant</li>
    <li>Bar</li>
</ul>
<ul><h4>Bronx</h4>
    <li>Pizza Place</li>
    <li>Bus Station</li>
    <li>Pharmacy</li>
</ul>
<ul><h4>Queens</h4>
    <li>Deli/Bodega</li>
    <li>Grocery Store</li>
    <li>Pizza Place</li>
</ul>
<ul><h4>Staten Island</h4>
    <li>Bus Stop</li>
    <li>Deli/Bodega</li>
    <li>Italian Restaurant</li>
</ul>
<ul><h4>Brooklyn</h4>
    <li>Pizza Place</li>
    <li>Deli/Bodega</li>
    <li>Bar</li>
</ul>

From the above information we can see that Pizza Places seem to be one of the most popular restaurants in New York, with it being popular in the Brooklyn, Queens and
Bronx areas. Further looking at the data reveals that it is the fourth most popular restuarant type in Staten Island as well. So we can be reasonably certain that pizza places are popular in New York city and this will be the restaurant that we will be recommending that our stakeholders open

In [179]:
Manhatan_pizza = Manhattan_venues.loc[Manhattan_venues['Venue Category']=='Pizza Place']
Brooklyn_pizza = Brooklyn_venues.loc[Brooklyn_venues['Venue Category']=='Pizza Place']
Bronx_pizza = Bronx_venues.loc[Bronx_venues['Venue Category']=='Pizza Place']
Queens_pizza = Queens_venues.loc[Queens_venues['Venue Category']=='Pizza Place']
Staten_pizza = Staten_venues.loc[Staten_venues['Venue Category']=='Pizza Place']


Manhatan_pizza.groupby(['Neighborhood', 'Venue Category']).size()

Neighborhood         Venue Category
Battery Park City    Pizza Place        4
Carnegie Hill        Pizza Place       10
Central Harlem       Pizza Place        2
Chelsea              Pizza Place        2
Chinatown            Pizza Place        2
Clinton              Pizza Place        2
East Harlem          Pizza Place        4
East Village         Pizza Place        8
Financial District   Pizza Place        8
Gramercy             Pizza Place        8
Greenwich Village    Pizza Place        2
Hamilton Heights     Pizza Place       10
Inwood               Pizza Place        6
Lenox Hill           Pizza Place       10
Little Italy         Pizza Place        4
Lower East Side      Pizza Place        6
Manhattan Valley     Pizza Place        6
Marble Hill          Pizza Place        2
Midtown              Pizza Place        2
Midtown South        Pizza Place        2
Morningside Heights  Pizza Place        2
Murray Hill          Pizza Place        4
Noho                 Pizza Place        

In [182]:
Bronx_pizza.groupby(['Neighborhood', 'Venue Category']).size()

Neighborhood         Venue Category
Allerton             Pizza Place        8
Baychester           Pizza Place        2
Bedford Park         Pizza Place        6
Belmont              Pizza Place       16
Bronxdale            Pizza Place        2
Castle Hill          Pizza Place        2
City Island          Pizza Place        2
Claremont Village    Pizza Place        4
Co-op City           Pizza Place        2
Concourse            Pizza Place        4
Concourse Village    Pizza Place        4
East Tremont         Pizza Place        8
Eastchester          Pizza Place        2
Edgewater Park       Pizza Place        4
Fordham              Pizza Place        8
High  Bridge         Pizza Place        6
Hunts Point          Pizza Place        2
Kingsbridge          Pizza Place       14
Kingsbridge Heights  Pizza Place       12
Melrose              Pizza Place       10
Morris Heights       Pizza Place        2
Morris Park          Pizza Place        8
Morrisania           Pizza Place        

In [183]:
Brooklyn_pizza.groupby(['Neighborhood', 'Venue Category']).size()

Neighborhood               Venue Category
Bath Beach                 Pizza Place        2
Bay Ridge                  Pizza Place       12
Bedford Stuyvesant         Pizza Place        4
Bensonhurst                Pizza Place        6
Boerum Hill                Pizza Place        2
Borough Park               Pizza Place        8
Brooklyn Heights           Pizza Place        6
Brownsville                Pizza Place        2
Bushwick                   Pizza Place        6
Carroll Gardens            Pizza Place       10
City Line                  Pizza Place        2
Clinton Hill               Pizza Place       10
Cobble Hill                Pizza Place        8
Coney Island               Pizza Place        2
Crown Heights              Pizza Place        6
Cypress Hills              Pizza Place        4
Ditmas Park                Pizza Place        4
Downtown                   Pizza Place        6
Dumbo                      Pizza Place        4
East New York              Pizza Place        

In [184]:
Queens_pizza.groupby(['Neighborhood', 'Venue Category']).size()

Neighborhood          Venue Category
Arverne               Pizza Place        2
Astoria               Pizza Place        4
Astoria Heights       Pizza Place        2
Bay Terrace           Pizza Place        2
Bayside               Pizza Place        6
Beechhurst            Pizza Place        2
Bellerose             Pizza Place        4
Broad Channel         Pizza Place        2
College Point         Pizza Place        6
Corona                Pizza Place        2
Edgemere              Pizza Place        4
Elmhurst              Pizza Place        2
Far Rockaway          Pizza Place        6
Floral Park           Pizza Place        2
Forest Hills          Pizza Place        4
Forest Hills Gardens  Pizza Place        2
Fresh Meadows         Pizza Place        2
Glen Oaks             Pizza Place        2
Glendale              Pizza Place        2
Hunters Point         Pizza Place        2
Jackson Heights       Pizza Place        4
Jamaica Center        Pizza Place        6
Kew Gardens      

In [187]:
Staten_pizza.groupby(['Neighborhood','Venue Category']).size()

Neighborhood       Venue Category
Annadale           Pizza Place       4
Arden Heights      Pizza Place       2
Arrochar           Pizza Place       4
Bulls Head         Pizza Place       8
Castleton Corners  Pizza Place       6
Charleston         Pizza Place       2
Clifton            Pizza Place       2
Dongan Hills       Pizza Place       4
Elm Park           Pizza Place       2
Eltingville        Pizza Place       6
Grant City         Pizza Place       4
Great Kills        Pizza Place       4
Greenridge         Pizza Place       2
Heartland Village  Pizza Place       2
New Dorp           Pizza Place       4
New Springville    Pizza Place       4
Old Town           Pizza Place       2
Pleasant Plains    Pizza Place       4
Prince's Bay       Pizza Place       4
Rosebank           Pizza Place       6
Rossville          Pizza Place       6
Shore Acres        Pizza Place       2
St. George         Pizza Place       2
Stapleton          Pizza Place       4
Tompkinsville      Pizza Place

In [174]:
Manhatan_pizza.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Arturo's,40.874412,-73.910271,Pizza Place
28,Chinatown,40.715618,-73.994279,Scarr's Pizza,40.715335,-73.991649,Pizza Place
145,Washington Heights,40.851903,-73.936900,Fresco's Pizzeria,40.855202,-73.937216,Pizza Place
211,Washington Heights,40.851903,-73.936900,Pizza Delia,40.855371,-73.933212,Pizza Place
242,Inwood,40.867684,-73.921210,Grandpa's Brick Oven Pizza,40.868215,-73.919419,Pizza Place


In [212]:
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)


for lat, lng, label in zip(Manhatan_pizza['Venue Latitude'], Manhatan_pizza['Venue Longitude'], Manhatan_pizza['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)
for lat, lng, label in zip(Bronx_pizza['Venue Latitude'], Bronx_pizza['Venue Longitude'], Bronx_pizza['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
for lat, lng, label in zip(Brooklyn_pizza['Venue Latitude'], Brooklyn_pizza['Venue Longitude'], Brooklyn_pizza['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='yellow',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york) 
for lat, lng, label in zip(Queens_pizza['Venue Latitude'], Queens_pizza['Venue Longitude'], Queens_pizza['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  
for lat, lng, label in zip(Staten_pizza['Venue Latitude'], Staten_pizza['Venue Longitude'], Staten_pizza['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='purple',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  

map_york

In [204]:
Manhattan_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
6607,Hudson Yards,40.756658,-74.000111,Silver Towers Dog Run,40.760854,-73.999765,Dog Run
6608,Hudson Yards,40.756658,-74.000111,Cachet Boutique Hotel,40.759773,-73.996460,Hotel
6609,Hudson Yards,40.756658,-74.000111,NYPD Mounted Unit,40.759155,-74.004121,Stables
6610,Hudson Yards,40.756658,-74.000111,NY Waterway 42nd St Bus,40.760050,-74.003379,Bus Station
6611,Hudson Yards,40.756658,-74.000111,City Lights Cruises,40.759804,-74.004025,Boat or Ferry


In [203]:
Brooklyn_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5615,Erasmus,40.646926,-73.948177,NSA Food Market,40.646610,-73.945609,Grocery Store
5616,Erasmus,40.646926,-73.948177,Met Food,40.647581,-73.943930,Supermarket
5617,Erasmus,40.646926,-73.948177,ideal Fresh Market,40.647974,-73.943826,Grocery Store
5618,Erasmus,40.646926,-73.948177,MTA Bus - B35/B35LTD - Church Ave & New York Ave,40.651014,-73.946691,Bus Line
5619,Erasmus,40.646926,-73.948177,King Liquors,40.651124,-73.947093,Liquor Store


In [202]:
Bronx_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2443,Kingsbridge Heights,40.870392,-73.901523,Carman Cafe,40.871159,-73.896762,Café
2444,Kingsbridge Heights,40.870392,-73.901523,Formaggio Pizza,40.873027,-73.905368,Pizza Place
2445,Kingsbridge Heights,40.870392,-73.901523,Kingsbridge Barber Shop,40.867590,-73.897748,Cosmetics Shop
2446,Kingsbridge Heights,40.870392,-73.901523,Caridad Spanish Restaurant,40.871832,-73.906700,Spanish Restaurant
2447,Kingsbridge Heights,40.870392,-73.901523,Lehman College Ballfield,40.871007,-73.895781,Baseball Field


In [201]:
Queens_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4227,Queensbridge,40.756091,-73.945631,Best Western Plaza Hotel,40.755829,-73.941065,Hotel
4228,Queensbridge,40.756091,-73.945631,Queensbridge Basketball Courts,40.755060,-73.949103,Basketball Court
4229,Queensbridge,40.756091,-73.945631,The Ravel Hotel Gym,40.753787,-73.948815,Athletics & Sports
4230,Queensbridge,40.756091,-73.945631,Profundo Pool Club,40.753719,-73.948878,Hotel Pool
4231,Queensbridge,40.756091,-73.945631,Track 114,40.753008,-73.947833,Platform


In [200]:
Staten_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1687,Richmond Valley,40.519541,-74.229571,Santander Bank,40.516317,-74.233371,Bank
1688,Fox Hills,40.617311,-74.081740,SUBWAY,40.618939,-74.082881,Sandwich Place
1689,Fox Hills,40.617311,-74.081740,Bums Pancake N Waffle Grill,40.618287,-74.085525,American Restaurant
1690,Fox Hills,40.617311,-74.081740,Targee Milk & Things,40.614410,-74.084455,Grocery Store
1691,Fox Hills,40.617311,-74.081740,MTA Bus - Tompkins Av & Vanderbilt Av (S52/S76...,40.620052,-74.077180,Bus Stop


In [188]:
Manhatan_pizza.groupby(['Venue Category']).size()

Venue Category
Pizza Place    152
dtype: int64

In [189]:
Brooklyn_pizza.groupby(['Venue Category']).size()

Venue Category
Pizza Place    268
dtype: int64

In [190]:
Bronx_pizza.groupby(['Venue Category']).size()

Venue Category
Pizza Place    198
dtype: int64

In [191]:
Queens_pizza.groupby(['Venue Category']).size()

Venue Category
Pizza Place    172
dtype: int64

In [192]:
Staten_pizza.groupby(['Venue Category']).size()

Venue Category
Pizza Place    98
dtype: int64

<h3>Pizza Place Analysis of New York</h3>
After analyzing the pizza places of New York City we find that of the five boroughs, Brooklyn has the most pizza places with 268 pizza places in the area, with the 
neighborhoods of Bay Ridge, Green Point and South Side having the most pizza places in the area, with each having 12. Areas such as Clinton Hill and Carrol Gardens
also have alot of pizza places with each having 10 in their area. The following list shows the other areas as well as the neighborhoods with highest number of pizza 
places:

<ul>
    <li>
        <ul><h4>Manhattan</h4> 152 pizza places
            <li>Carnegie Hill 10</li>
            <li>Hamilton Heights 10</li>
            <li>Lenox Hill 10</li>  
        </ul>
    </li>
    <li>
         <ul><h4>Bronx</h4>198 pizza places
             <li>Belmot 16</li>
             <li>Kingsbridge 14</li>
             <li>Melrose 10</li>
        </ul>
    </li>
    <li>
         <ul><h4>Brooklyn</h4>268 pizza places
            <li>Bay Ridge 12</li>
            <li>South Side 12</li>
            <li>Green Point 12</li>
        </ul>
    </li>
    <li>
         <ul><h4>Queens</h4>172 pizza places
            <li>Sunnyside 10</li>
            <li>Sunnyside Gardens 10</li>
            <li>Long Island City 8</li>
        </ul>
    </li>
    <li>
         <ul><h4>Staten Island</h4>98 pizza places
            <li>Bulls Head 8</li>
            <li>Castle Corners 6</li>
            <li>Eltingville 6</li>
        </ul>
    </li>
</ul>
    
    
Looking at the above information we might want to open a pizza place in the Brooklyn area, however, when we take into account the percentage of pizza places of the total
number of venues within an area, we find that pizza places only make up for 4.8% of Brooklyn venues, 2.3% of Manhattan venues, 4% of Queens venues, 5.8% of Staten Island venues and 8% of Bronx venues.
    
Taking this into account we can assume pizza places generally do better in the Bronx, and specifically in the Belmot, Kingsbridge and Melrose.

<h3>Conclusion</h3>
I would suggest opening a Pizza Place in the Bronx in either Belmot, Kingsbridge or Melrose.


In [205]:
map_york = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(Bronx_pizza['Venue Latitude'], Bronx_pizza['Venue Longitude'], Bronx_pizza['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7,
        parse_html=False).add_to(map_york)  


map_york